In [1]:
%reload_ext autoreload
%autoreload 
import sys, os
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root_path not in sys.path:
    sys.path.append(project_root_path)
print(project_root_path)

import datetime as dt
import numpy as np
import pandas as pd
import plotly.express as px

import pymd 
import qpt_stress_test.services.positions as positions
import qpt_stress_test.core.config as config
import qpt_stress_test.db.tasks as db_tasks
import qpt_stress_test.db.repositories.drivers.pyodbc as pyodbc
import qpt_stress_test.db.repositories.drivers.sqlalchemy as sqlalchemy
import qpt_stress_test.db.repositories.qpt_pg as qpt_pg
import qpt_stress_test.db.repositories.qpt_mssql as qpt_mssql
import qpt_stress_test.db.repositories.databricks as db_trading
from qpt_stress_test.db.tasks import bfc_rds_sqlalchemy_engine_factory, sv_awoh_dw01_pyodbc_connection_factory,  gdt_cluster_databricks_connection_factory

from importlib import reload
reload(positions)
reload(config)
reload(db_tasks)

pymd.enable_logging()

c:\Users\skingham\Projects\galaxysk\qpt_stress_test


In [2]:
# Services
trade_service = positions.TradeService()


### Get all symbols and for the period from the beginning of the historic period

In [3]:
reporting_dt = dt.date.today() - dt.timedelta(days=1)
hist_start_dt = reporting_dt - dt.timedelta(days=365)
report_utc_dtt = dt.datetime(year=2022, month=10, day=7, hour=22, minute=0, second=0)



In [4]:
cme_positions = trade_service.cme_positions(report_utc_dtt)
cme_positions_columns, cme_positions_data = cme_positions.as_list()
cme_positions_map = cme_positions.as_instrument_map()
cme_positions_df = cme_positions.as_dataframe()
display(cme_positions_df)

c:\Users\skingham\Projects\galaxysk\qpt_stress_test\qpt_stress_test\db\repositories\drivers\pyodbc.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self._query, con=self._pyodbc_conn_factory())


,eod_date,as_of,exchange,account,instrument_type,instrument,expiration_time,is_linear,underlying,price,position,notional,unrealized_pnl,EntryCost
0,2022-10-07,2022-10-07 07:30:05.797,ED&F,UNMBF222,CME FUTURE,ETHX2,2022-11-26,1,ETH,1323.5,12,794100.0,-100.0,794200.0
1,2022-10-07,2022-10-07 15:56:59.380,ED&F,UNMBF222,CME FUTURE,ETHV2,2022-10-29,1,ETH,1324.5,382,25297950.0,1563345.0,23734605.0
2,2022-10-07,2022-10-07 15:28:03.013,ED&F,UNMBF222,CME FUTURE,BTCX2,2022-11-26,1,BTC,19420.0,37,3592700.0,240925.0,3351775.0
3,2022-10-07,2022-10-07 15:56:59.160,ED&F,UNMBF222,CME FUTURE,BTCV2,2022-10-29,1,BTC,19410.0,427,41440350.0,2774875.0,38665475.0


In [5]:
contracts = trade_service.active_contracts(hist_start_dt)
contracts_columns, contracts_data = contracts.as_list()
contracts_map = contracts.as_instrument_map()
contracts_df = contracts.as_dataframe()

display(contracts_df)

,symbol_bfc,symbol_exch,exchange,endpoint,instrument_type,expiration_time,is_linear,contract_size,qty_multiplier,currency_position,currency_settlement,underlying
0,H_P_ANKR-USD-SWAP,ANKR-USD,HUOBI,HUBI,SWAP,2099-12-31 23:00:00,0,10.00,1,ANKR,ANKR,ANKR
1,H_P_AVAX-USD-SWAP,AVAX-USD,HUOBI,HUBI,SWAP,2099-12-31 08:00:00,0,10.00,1,,AVAX,AVAX
2,BCH221111,BCH221111,HUOBI,FUBI,FUTURE,2022-11-11 08:00:00,0,10.00,1,,BCH,BCH
3,BLUNABUSD,LUNABUSD,BINANCE,BINE,SWAP,2099-12-31 08:00:00,1,NaN,1000000000,LUNA,BUSD,LUNA
4,H_P_LEND-USD-SWAP,LEND-USD,HUOBI,HUBI,SWAP,2099-01-01 00:00:00,0,NaN,1,LEND,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3883,R_T_NEAR_USDC_SWAP,NEAR_USDC-PERPETUAL,DERIBIT,DERI,SWAP,3000-01-01 08:00:00,1,1.00,1000000000,NEAR,USDC,NEAR
3884,R_T_TRX_USDC_SWAP,TRX_USDC-PERPETUAL,DERIBIT,DERI,SWAP,3000-01-01 08:00:00,1,100.00,1000000000,TRX,USDC,TRX
3885,R_T_DOGE_USDC_SWAP,DOGE_USDC-PERPETUAL,DERIBIT,DERI,SWAP,3000-01-01 08:00:00,1,100.00,1000000000,DOGE,USDC,DOGE
3886,R_T_BCH_USDC_SWAP,BCH_USDC-PERPETUAL,DERIBIT,DERI,SWAP,3000-01-01 08:00:00,1,0.02,1000000000,BCH,USDC,BCH


In [6]:
import pymd
# pymd.init() # pymd.init() will load most recent symbols
pymd.instrument_service.reload()


2023-01-25 09:59:24,797-pymd.instrument INFO reloaded 3867 instruments(4.060 sec)


In [7]:
binance_instruments = pymd.instrument_service.get_instruments('binance')
# get Binance spot instrument, string parameter
huobi_spot_instruments = pymd.instrument_service.get_instruments('huobi', 'spot')
# get both huobi usdt + coin perpetual, enum parameter
huobi_swap_instruments = pymd.instrument_service.get_instruments('huobi', pymd.InstrumentType.SWAP)

exchange_to_endpoint_map = pymd.instrument_service.get_exchange_to_endpoint_mapping()
print(exchange_to_endpoint_map)
#instruments = pymd.instrument_service.get_instrument(exchange_or_endpoint: str, bfc_symbol_name: str) -> Instrument
#instruments = pymd.instrument_service.get_instrument(exchange_or_endpoint: str, inst_type: InstrumentType, exchange_symbol_name: ) -> Instrument:


{'OKEX': 'OKEX', 'CME': 'CME', 'WOO': 'WOOX', 'BITSTAMP': 'BTSE', 'BULLISH': 'BULL', 'BINANCE': 'BINE', 'HUOBI': 'HUBI', 'KRAKEN': 'KRKE', 'LMAXDIGITAL': 'LMAC', 'GATE': 'GATE', 'DYDX': 'DYDX', 'DERIBIT': 'DERI', 'COINBASE_PRO': 'GDAX', 'LMAX': 'LMAX', 'BINANCE_US': 'BINU'}


In [8]:
import clickhouse_driver
from datetime import datetime, timedelta
ck_client = clickhouse_driver.Client(host=config.CLICKHOUSE_HOST 
                                     , user='reader1'
                                     , compression=False
                                     , send_receive_timeout=600
                                     , settings={'use_numpy': True , 'strings_encoding': 'ascii'})

ret, cols = ck_client.execute("""
select symbol, name, volume
from cex_data.coin_marketcap 
order by `rank` asc
limit 10
""", columnar=False, with_column_types=True)
coin_marketcap_df = pd.DataFrame(ret, columns=[tuple[0] for tuple in cols])
display(coin_marketcap_df)

EOFError: Unexpected EOF while reading bytes

In [ ]:
import plotly.express as px
import pandas as pd
import clickhouse_driver
from datetime import datetime, timedelta
time_point = datetime(2022, 8, 18, 20, 15, 0)
time_lookahead = time_point - timedelta(minutes=1)
ck_client = clickhouse_driver.Client(config.CLICKHOUSE_HOST
                                     , user='reader1'
                                     , compression=False
                                     , send_receive_timeout=600
                                     , settings={'use_numpy': True , 'strings_encoding': 'ascii'})
_sql = f"""
select strike, delta, dateDiff('day', parseDateTimeBestEffortOrNull('{time_point.strftime('%Y-%m-%d %H:%M:%S')}'), expiration) as days_exp, mark_iv 
from cex_data.deribit_option_snapshot_btc t1
inner join  (select symbol, max(`timestamp`) as mt
from cex_data.deribit_option_snapshot_btc
where `timestamp` BETWEEN '{time_lookahead.strftime('%Y-%m-%d %H:%M:%S')}' and '{time_point.strftime('%Y-%m-%d %H:%M:%S')}'
and dateDiff('day', parseDateTimeBestEffortOrNull('{time_point.strftime('%Y-%m-%d %H:%M:%S')}'), expiration) >= 3
and delta >= 0.04 and strike <= 60000
group by symbol ) t2 on t1.symbol = t2.symbol and t1.`timestamp` = t2.mt
order by strike, expiration 
"""
print(_sql)
ret, cols = ck_client.execute(_sql, types_check=False, columnar=False, with_column_types=True)
df = pd.DataFrame(ret, columns=[tuple[0] for tuple in cols])
fig = px.line_3d(df, x="delta", y="days_exp", z="mark_iv", color='days_exp', title =f"BTC Vol Surface at {time_lookahead.strftime('%Y-%m-%d %H:%M:%S')}")
fig.update_layout(width=1200, height=700)
fig.show()

In [ ]:
# By column 
ret, col = ck_client.execute("""
select symbol, name, volume
from cex_data.coin_marketcap 
order by `rank` asc
limit 10
""", columnar=True, with_column_types=True)
data = {}
for i in range(len(col)):
    data[col[i][0]] = ret[i]
df = pd.DataFrame(data=data)

In [ ]:
"""
with toStartOfInterval(`timestamp`, INTERVAL '1 hour') as frame
select frame, any(price) as open_price
from mark_prices.coin_metrics_reference_rate
where name = 'BTC'
and `timestamp` BETWEEN '2022-08-10 21:00:00' and '2022-08-20 21:00:00' 
group by frame
order by frame asc
"""


In [ ]:
"""
with ohlc as (
select any(price) as open_px, max(price) as high_px, min(price) as low_px, anyLast(price) as close_px
from mark_prices.coin_metrics_reference_rate
where name = 'BTC'
and `timestamp` BETWEEN '2022-06-08 21:00:00' and '2022-06-21 21:00:00' 
group by toStartOfInterval(`timestamp`, INTERVAL '1 day')
order by toStartOfInterval(`timestamp`, INTERVAL '1 day') asc
), (select COUNT() from ohlc) as T
select 
sqrt(
	1/(2*T)*sum(
			ln(high_px/low_px) * ln(high_px/low_px)
						-  (2*ln(2)-1) / T * ln(close_px/open_px) * ln(close_px/open_px)) )
from ohlc
"""

In [ ]:

"""
with t1 as (
select toStartOfInterval(`timestamp`, INTERVAL '1 hour') as tm1, any(price) as px_btc
from mark_prices.coin_metrics_reference_rate
where name = 'BTC'
and `timestamp` BETWEEN '2022-05-14 00:00:00' and '2022-05-20 00:00:00'
group by tm1
order by tm1 desc
), t2 as (
  select toStartOfInterval(`timestamp`, INTERVAL '1 hour') as tm2, any(price) as px_eth
  from mark_prices.coin_metrics_reference_rate
  where name = 'ETH'
  and `timestamp` BETWEEN '2022-05-14 00:00:00' and '2022-05-20 00:00:00'
  group by tm2
  order by tm2 desc
), t3 as (
  select toStartOfInterval(`timestamp`, INTERVAL '1 hour') as tm3, any(price) as px_sol
  from mark_prices.coin_metrics_reference_rate
  where name = 'SOL'
  and `timestamp` BETWEEN '2022-05-14 00:00:00' and '2022-05-20 00:00:00'
  group by tm3
  order by tm3 desc
)
select t1.px_btc, t2.px_eth, t3.px_sol
from t1
join t2 on t1.tm1 = t2.tm2
join t3 on t1.tm1 = t3.tm3
"""

In [ ]:
start_dt = pymd.ChicagoTimeZone.localize(dt.datetime(year=2022, month=1, day=1, hour=0, minute=0))
end_dt = pymd.ChicagoTimeZone.localize(dt.datetime(year=2022, month=1, day=10, hour=0, minute=0))
# df = pymd.data_access.coinmetrics_reference_rate(['BTC'], start_dt, end_dt, '1 day', 'open')
# print(pymd.data_access._endpoint2table)
df = pymd.data_access.future_mark_prices('BINE', ['BTCUSD_221230', 'BTCUSD_230331'], start_dt, end_dt) #, sampling_timeframe='1 day', aggregation_function='close')
print(df.shape)
display(df)